In [10]:
%reload_ext autoreload
%autoreload 2

import os
import json
import re

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("OpenAI.env")

client = OpenAI()

# DIR = "output"
DIR = "output_DataScience"

In [11]:
#Images_Intro  creation
from typing import List, Any, Optional

from ImageAgent import Slide, ImagePromptAgent, OpenAIAgentConfig

def process_intro_videos(image_intro) -> None:

    agent = ImagePromptAgent(client, OpenAIAgentConfig(
                                        model="gpt-5-nano",      # текстова модель для створення промта
                                        image_model="gpt-image-1" # модель для рендеру зображення (якщо потрібно)
                                        ))

    title = COURSE_DATA.get('course_name', '') + ' ' + image_intro
    bullets = COURSE_DATA.get('description', '')
    out_dir = COURSE_DATA.get('path', 'Untitled')
    
    print(out_dir)
    print(f"Creating intro content: {title}")
    print(f"Description: {bullets}")

            
    prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
    # print(prompt)
    #Image generation
    save_path = f"{image_intro}.png"
    agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

In [12]:
#Images content creation
from typing import List, Any, Optional

from ExtractSlidesContentAgent import LectureSlidesAgent
from ImageAgent import Slide, ImagePromptAgent, OpenAIAgentConfig

def process_module_images(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    ) -> None:

    agent = ImagePromptAgent(client, OpenAIAgentConfig(
                                        model="gpt-5-nano",      # текстова модель для створення промта
                                        image_model="gpt-image-1" # модель для рендеру зображення (якщо потрібно)
                                        ))
    
    if not items:
        print("  No videos found.")
        return
    
    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating images for video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "image_out")
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)
            if slides_list == []:
                s = [i for i in range(2, content_agent.slides_count()+1)] 
            else:
                s = slides_list

            print("Creating images for slides ", s)
            
            for idx in s:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
            
                print("=" * 80 )
                print(f"Slide {idx}")
                print(f"Title: {title}")
                print("Bullets:")
                if bullets:
                    for b in bullets:
                        print(f" - {b}")
                else:
                    print(" - (no bullets found)")
                #prompt for image generation
                prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
                # print(prompt)
                #Image generation
                save_path = f"Slide_{idx}.png"
                agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

            
    else: #Videos for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "image_out")
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            s = [i for i in range(2, content_agent.slides_count()+1)] 
            print("Creating images for slides ", s)
            
            for idx in s:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
            
                print("=" * 80 )
                print(f"Slide {idx}")
                print(f"Title: {title}")
                print("Bullets:")
                if bullets:
                    for b in bullets:
                        print(f" - {b}")
                else:
                    print(" - (no bullets found)")
                #prompt for image generation
                prompt = agent.generate_prompt(Slide(title=title, bullets= bullets))
                # print(prompt)
                #Image generation
                save_path = f"Slide_{idx}.png"
                agent.render_image(prompt, size="1536x1024", out_dir=out_dir, save_path=save_path)

In [13]:
#Audio content creation
from typing import List, Any, Optional

from VoiceAgent import PresentationVoiceAgent
from ExtractSlidesContentAgent import LectureSlidesAgent

def process_module_audio(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    ) -> None:

    instructions = (
        "Read the text with a tone that is clear, professional, high energy and confident, but not monotonous. "
        "Add slight warmth and emphasis so learners stay interested. "
        "Use natural pauses and vary your sentence rhythm to keep the narration dynamic. "
        "Avoid sounding like you’re just reading; instead, guide the learner as if you are teaching them step by step. "
        "Keep the delivery professional but conversational, as if you’re explaining important ideas to someone who wants to understand, not just memorize."
    )

    if not items:
        print("  No videos found.")
        return
    
    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating audio for video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            # out_dir = os.path.join(json_path, "audio_out")
    
            agent = PresentationVoiceAgent(client,
                                           model="tts-1",
                                           voice="nova",
                                           out_dir=out_dir,
                                           instructions=instructions,
                                           verbose=False)

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)

            if slides_list == []:
                s = [i for i in range(1, content_agent.slides_count()+1)] 
            else:
                s = slides_list

            print("Creating audio for slides ", s)
            
            for idx in s:
                notes = content_agent.get_notes(idx)            
                # print("=" * 80 )
                # print(f"Slide {idx}")
                # print(f"Title: {title}")
                # print(f"Notes: {notes}")
                # print("Bullets:")
                # if bullets:
                #     for b in bullets:
                #         print(f" - {b}")
                # else:
                #     print(" - (no bullets found)")
                #prompt for audio generation
                agent.process(idx, notes)

            
    else: #Audio for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "audio_out")
    
            agent = PresentationVoiceAgent(client,
                                           model="tts-1",
                                           voice="nova",
                                           out_dir=out_dir,
                                           instructions=instructions,
                                           verbose=False)

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            print(os.path.join(json_path, "content.json"))
            
            #slice for rebuild!!!!
            # print("Slides list", slides_list)

            s = [i for i in range(1, content_agent.slides_count()+1)] 
            print("Creating audio for slides ", s)
            
            for idx in s:
                notes = content_agent.get_notes(idx)            
                # print("=" * 80 )
                # print(f"Slide {idx}")
                # print(f"Title: {title}")
                # print(f"Notes: {notes}")
                # print("Bullets:")
                # if bullets:
                #     for b in bullets:
                #         print(f" - {b}")
                # else:
                #     print(" - (no bullets found)")
                
                # prompt for audio generation
                agent.process(idx, notes)


In [14]:
#Pptx content creation
from typing import List, Any, Optional

from ExtractSlidesContentAgent import LectureSlidesAgent
from PptxCopyAndFillAgent import PptxSlideCopyAgent

def process_module_pptx(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    ) -> None:

    if not items:
        print("  No videos found.")
        return
    
    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating PPTX for video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            pptx_template_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'template.pptx')
            pptx_output = os.path.join(json_path, "content.pptx")
            image_dir = os.path.join(json_path, "image_out")
            bg_intro_image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Intro.png')
            bg_outro_image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Outro.png')

            agent = PptxSlideCopyAgent(
                    pptx_path=pptx_template_path,
                    out_path=pptx_output,
                    src_slide_index=1,                  # копіюємо 2-й слайд
                    insert_after_src=True
                )

            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            result = agent.run_intro(
                title_text=COURSE_DATA.get('course_name', 'Untitled'),
                body_text=it.get('title', 'Untitled'),
                notes_text=content_agent.get_notes(1), 
                bg_image_path=bg_intro_image_path)

            result = agent.run_intro(
                title_text=None,
                body_text=None,
                notes_text=None, 
                bg_image_path=bg_outro_image_path,
                intro = "outro")

            s = [i for i in range(2, content_agent.slides_count()+1)]
            for idx in s[::-1]:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
                notes = content_agent.get_notes(idx)
                image = f"{image_dir}/Slide_{idx}.png" #!!!!!!! Check this code
                
                result = agent.run(
                        bg_image_path=image,          # фон (None, якщо не потрібно)
                        title_text=title,
                        bullets=bullets,
                        notes_text=notes
                )
            
            result = agent.delete_slide(1)
            # agent.autosize_text_boxes() 
            print(f"Presentation {result} created!")
                 
    else: #Audio for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating PPTX for Video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            pptx_template_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'template.pptx')
            pptx_output = os.path.join(json_path, "content.pptx")
            image_dir = os.path.join(json_path, "image_out")
            bg_intro_image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Intro.png')
            bg_outro_image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Outro.png')

            agent = PptxSlideCopyAgent(
                    pptx_path=pptx_template_path,
                    out_path=pptx_output,
                    src_slide_index=1,                  # копіюємо 2-й слайд
                    insert_after_src=True
                )

            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )
            result = agent.run_intro(
                title_text=COURSE_DATA.get('course_name', 'Untitled'),
                body_text=it.get('title', 'Untitled'),
                notes_text=content_agent.get_notes(1), 
                bg_image_path=bg_intro_image_path)

            result = agent.run_intro(
                title_text=None,
                body_text=None,
                notes_text=None, 
                bg_image_path=bg_outro_image_path,
                intro = "outro")

            s = [i for i in range(2, content_agent.slides_count()+1)]
            for idx in s[::-1]:
                title = content_agent.get_title(idx)
                bullets = content_agent.get_content(idx)
                notes = content_agent.get_notes(idx)
                image = f"{image_dir}/Slide_{idx}.png" #!!!!!!! Check this code
                
                result = agent.run(
                        bg_image_path=image,          # фон (None, якщо не потрібно)
                        title_text=title,
                        bullets=bullets,
                        notes_text=notes
                )
            
            result = agent.delete_slide(1)
            # agent.autosize_text_boxes() 
            print(f"Presentation {result} created!")


In [32]:
#Video content creation
from typing import List, Any, Optional
from ExtractSlidesContentAgent import LectureSlidesAgent
from VideoAgent import VideoAgent, Typography, BackgroundPlates, Layout, Anim, Subtitles

def process_module_video(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    ) -> None:

    if not items:
        print("  No videos found.", item_index)
        return
    

    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating video for video content: {it.get('title', 'Untitled')}")
           
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "video_out")
            image_dir = os.path.join(json_path, "image_out")
            audio_dir = os.path.join(json_path, "audio_out")

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )

            agent = VideoAgent(
                out_dir=out_dir,
                typography=Typography(
                    font="Inter-Bold",     # шрифт
                    title_size=60,         # РОЗМІР ЗАГОЛОВКА
                    subtitle_size=40,      # РОЗМІР ПІДЗАГОЛОВКА
                    title_color="gray",
                    subtitle_color="black",
                    stroke_color="#111111",
                    stroke_width=1,
                    title_method = "label",     # label для того щоб не переносилось
                    subtitle_method = "caption"
                ),
                plates=BackgroundPlates(
                    enabled=True,          # ← «ґрунт/плашка»: вкл/викл
                    opacity=0.6,           # прозорість
                    padding_left=40, padding_right=40,
                    padding_top=28, padding_bottom=16,
                    title_radius=24, subtitle_radius=18,
                ), 
                layout=Layout(
                    size=(1280, 720),     # портретне відео (Reels/Shorts)
                    title_left_margin=0.08,
                    subtitle_left_margin=0.08,
                    subtitle_bottom_margin=0.09,
                    subtitle_width_ratio=0.70,  # ширина колонки підзаголовка
                ),
                anim=Anim(
                    intro_fade=1.2,
                    outro_start=-2.5,
                    outro_fade=1.4,
                ),
            )
            
            #slice for rebuild!!!!
            print("Slides list", slides_list)   
            if slides_list == []:
                s = [i for i in range(2, content_agent.slides_count()+1)] 
            else:
                s = slides_list
            print("Creating video for slides ", s)
            
            for idx in s:          
                print("=" * 80 )
                
                image_path = f"{image_dir}/Slide_{idx}.png"
                title_text = content_agent.get_title(idx)
                title_audio_path = f"{audio_dir}/Slide_{idx}_000_Intro.mp3"
                notes = content_agent.get_notes(idx)
                sentences = [p.strip() for p in notes.split("\n") if p.strip()]
                
                title_subs_text = sentences[0]              
                content = content_agent.get_content(idx)

                bullets = []
                if content:
                    for i, s in enumerate(sentences[1:]):
                        v = {}
                        try:
                            v["text"] = content[i]
                        except:
                            v["text"] = ""
                        v["audio"] = f"{audio_dir}/Slide_{idx}_Bullet_{(i+1):03d}.mp3"
                        v["subs"] = s
                        bullets.append(v)
                    
                    # print(bullets)
                else:
                    print(" - (no content found)")
                    
                output_path=f"slide_{idx}.mp4"

                # prompt for audio generation
                agent.create_video_bulleted_sequence(
                    image_path=image_path,
                    title_text=title_text,
                    title_audio_path=title_audio_path,
                    title_subs_text=title_subs_text,
                    bullets=bullets,
                    output_path=output_path,
                    burn_subtitles=False,   # True, якщо хочеш підпалити субтитри
                )

            
    else: #Video for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "video_out")
            image_dir = os.path.join(json_path, "image_out")
            audio_dir = os.path.join(json_path, "audio_out")

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )

            agent = VideoAgent(
                out_dir=out_dir,
                typography=Typography(
                    font="Inter-Bold",     # шрифт
                    title_size=60,         # РОЗМІР ЗАГОЛОВКА
                    subtitle_size=40,      # РОЗМІР ПІДЗАГОЛОВКА
                    title_color="gray",
                    subtitle_color="black",
                    stroke_color="#111111",
                    stroke_width=1,
                    title_method = "label",     # label для того щоб не переносилось
                    subtitle_method = "caption"
                ),
                plates=BackgroundPlates(
                    enabled=True,          # ← «ґрунт/плашка»: вкл/викл
                    opacity=0.6,           # прозорість
                    padding_left=40, padding_right=40,
                    padding_top=28, padding_bottom=16,
                    title_radius=24, subtitle_radius=18,
                ), 
                layout=Layout(
                    size=(1280, 720),     # портретне відео (Reels/Shorts)
                    title_left_margin=0.08,
                    subtitle_left_margin=0.08,
                    subtitle_bottom_margin=0.09,
                    subtitle_width_ratio=0.70,  # ширина колонки підзаголовка
                ),
                anim=Anim(
                    intro_fade=1.2,
                    outro_start=-2.5,
                    outro_fade=1.4,
                ),
            )
            
            #slice for rebuild!!!!
            print("Slides list", slides_list)   
            if slides_list == []:
                s = [i for i in range(2, content_agent.slides_count()+1)] 
            else:
                s = slides_list
            print("Creating video for slides ", s)
            
            for idx in s:          
                print("=" * 80 )
                
                image_path = f"{image_dir}/Slide_{idx}.png"
                title_text = content_agent.get_title(idx)
                title_audio_path = f"{audio_dir}/Slide_{idx}_000_Intro.mp3"
                notes = content_agent.get_notes(idx)
                sentences = [p.strip() for p in notes.split("\n") if p.strip()]
                
                title_subs_text = sentences[0]              
                content = content_agent.get_content(idx)

                bullets = []
                if content:
                    for i, s in enumerate(sentences[1:]):
                        v = {}
                        try:
                            v["text"] = content[i]
                        except:
                            v["text"] = ""
                        v["audio"] = f"{audio_dir}/Slide_{idx}_Bullet_{(i+1):03d}.mp3"
                        v["subs"] = s
                        bullets.append(v)
                    
                    # print(bullets)
                else:
                    print(" - (no content found)")
                    
                output_path=f"slide_{idx}.mp4"

                # prompt for audio generation
                agent.create_video_bulleted_sequence(
                    image_path=image_path,
                    title_text=title_text,
                    title_audio_path=title_audio_path,
                    title_subs_text=title_subs_text,
                    bullets=bullets,
                    output_path=output_path,
                    burn_subtitles=False,   # True, якщо хочеш підпалити субтитри
                )


In [33]:
#Video final content creation
from typing import List, Any, Optional
from ExtractSlidesContentAgent import LectureSlidesAgent
from VideoAgent import VideoAgent, Typography, BackgroundPlates, Layout, Anim, Subtitles

def process_module_video_final(
    module: dict, 
    items: List[dict], 
    item_index: Optional[int] = None,
    video_final: Optional[str] = None, #  Inrtro, Outro, Final створення картинки початковий чи кінцевий 
    ) -> None:

    if not items:
        print("  No videos found.", item_index)
        return
    

    if item_index is not None: #One Video
        if 1 <= item_index <= len(items):
            it = items[item_index - 1]
            print(f"Creating video for video content: {it.get('title', 'Untitled')}")
            print("Creating:", video_final)
           
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "video_out")
            image_dir = os.path.join(json_path, "image_out")
            audio_dir = os.path.join(json_path, "audio_out")

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )

            agent = VideoAgent(
                out_dir=out_dir,
                typography=Typography(
                    font="Inter-Bold",     # шрифт
                    title_size=60,         # РОЗМІР ЗАГОЛОВКА
                    subtitle_size=40,      # РОЗМІР ПІДЗАГОЛОВКА
                    title_color="gray",
                    subtitle_color="black",
                    stroke_color="#111111",
                    stroke_width=1,
                    title_method = "label",     # label для того щоб не переносилось
                    subtitle_method = "caption"
                ),
                plates=BackgroundPlates(
                    enabled=True,          # ← «ґрунт/плашка»: вкл/викл
                    opacity=0.6,           # прозорість
                    padding_left=40, padding_right=40,
                    padding_top=28, padding_bottom=16,
                    title_radius=24, subtitle_radius=18,
                ), 
                layout=Layout(
                    size=(1280, 720),     # портретне відео (Reels/Shorts)
                    title_left_margin=0.08,
                    subtitle_left_margin=0.08,
                    subtitle_bottom_margin=0.09,
                    subtitle_width_ratio=0.70,  # ширина колонки підзаголовка
                ),
                anim=Anim(
                    intro_fade=1.2,
                    outro_start=-2.5,
                    outro_fade=1.4,
                ),
            )
            
            print("=" * 80 )
            if video_final=="Intro":
                image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Intro.png')
                text_blocks = [COURSE_DATA.get('course_name', 'Untitled'), it.get('title', 'Untitled')]
                audio_path = f"{audio_dir}/Slide_1_000_Intro.mp3"
                output_path = "intro.mp4"
                subtitles = content_agent.get_notes(1)
                agent.create_into_video(
                    image_path=image_path,
                    text_blocks=text_blocks,
                    audio_path=audio_path,
                    output_path=output_path,
                    subtitles=Subtitles(text=subtitles, burn_in=False),
                )           
            elif video_final=="Outro":
                agent.typography.title_method = "label"
                image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Outro.png')
                text_blocks = ["Thank you!", "See you on next lecture"]
                output_path = "outro.mp4"
                agent.create_outro_video(
                    image_path=image_path,
                    text_blocks=text_blocks,
                    output_path=output_path,
                    duration=6.0,                 # рівно стільки триватиме відео
                    create_empty_srt=True,        # створить пустий SRT
                                )
            elif video_final=="Final":
                agent.create_final_video(
                                    output_dir=json_path,
                                    video_name="video.mp4",
                                    srt_name="video.srt"
                                )
                
    else: #Video for whole module
        for i, it in enumerate(items, 1):
            print(f"Creating video content: {it.get('title', 'Untitled')}")
            print("Creating:", video_final)
            
            json_path = it.get('path', 'Untitled')
            out_dir = os.path.join(json_path, "video_out")
            image_dir = os.path.join(json_path, "image_out")
            audio_dir = os.path.join(json_path, "audio_out")

            
            content_agent = LectureSlidesAgent(os.path.join(json_path, "content.json") )

            agent = VideoAgent(
                out_dir=out_dir,
                typography=Typography(
                    font="Inter-Bold",     # шрифт
                    title_size=60,         # РОЗМІР ЗАГОЛОВКА
                    subtitle_size=40,      # РОЗМІР ПІДЗАГОЛОВКА
                    title_color="gray",
                    subtitle_color="black",
                    stroke_color="#111111",
                    stroke_width=1,
                    title_method = "label",     # label для того щоб не переносилось
                    subtitle_method = "caption"
                ),
                plates=BackgroundPlates(
                    enabled=True,          # ← «ґрунт/плашка»: вкл/викл
                    opacity=0.6,           # прозорість
                    padding_left=40, padding_right=40,
                    padding_top=28, padding_bottom=16,
                    title_radius=24, subtitle_radius=18,
                ), 
                layout=Layout(
                    size=(1280, 720),     # портретне відео (Reels/Shorts)
                    title_left_margin=0.08,
                    subtitle_left_margin=0.08,
                    subtitle_bottom_margin=0.09,
                    subtitle_width_ratio=0.70,  # ширина колонки підзаголовка
                ),
                anim=Anim(
                    intro_fade=1.2,
                    outro_start=-2.5,
                    outro_fade=1.4,
                ),
            )
            
            print("=" * 80 )
            if video_final=="Intro":
                image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Intro.png')
                text_blocks = [COURSE_DATA.get('course_name', 'Untitled'), it.get('title', 'Untitled')]
                audio_path = f"{audio_dir}/Slide_1_000_Intro.mp3"
                output_path = "intro.mp4"
                subtitles = content_agent.get_notes(1)
                agent.create_into_video(
                    image_path=image_path,
                    text_blocks=text_blocks,
                    audio_path=audio_path,
                    output_path=output_path,
                    subtitles=Subtitles(text=subtitles, burn_in=False),
                )           
            elif video_final=="Outro":
                agent.typography.title_method = "label"
                image_path = os.path.join(COURSE_DATA.get('path', 'Untitled'), 'Outro.png')
                text_blocks = ["Thank you!", "See you on next lecture"]
                output_path = "outro.mp4"
                agent.create_outro_video(
                    image_path=image_path,
                    text_blocks=text_blocks,
                    output_path=output_path,
                    duration=6.0,                 # рівно стільки триватиме відео
                    create_empty_srt=True,        # створить пустий SRT
                )            
            elif video_final=="Final":
                agent.create_final_video(
                                    output_dir=json_path,
                                    video_name="video.mp4",
                                    srt_name="video.srt"
                                )

In [ ]:
import json
import os
from typing import List, Any, Optional

# ==============================================================
# НАЛАШТУВАННЯ
# ==============================================================
# JSON_FILE = "AML_Uncovered_curriculum_with_paths.json"
JSON_FILE = "DataScience_curriculum_with_paths.json"

# ==============================================================
# 1. ЗАВАНТАЖЕННЯ ДАНИХ
# ==============================================================
json_file_with_path = os.path.join(DIR, JSON_FILE)
with open(json_file_with_path, "r", encoding="utf-8") as f:
    COURSE_DATA = json.load(f)

# ==============================================================
# ГЛОБАЛЬНІ КОНСТАНТИ
# ==============================================================
MODULE_CONTENT_KEYS = ["videos", "readings", "case_studies", "quiz"]
EXAM_CONTENT_KEYS = ["case_studies", "quiz"]

# ==============================================================
# 4. МЕНЕДЖЕР ДЛЯ МОДУЛІВ
# ==============================================================
def content_manager(
    module_num: int,
    content_key: str,
    item_index: Optional[int] = None,
    slides_list: Optional[List] = [],
    video_final: Optional[str] = None, #  Inrtro, Outro, Final створення картинки початковий чи кінцевий 
    content_type: Optional[List] = ['Image', 'Audio', 'pptx', 'Video'],  
) -> None:

    if content_key not in MODULE_CONTENT_KEYS:
        print(f"ERROR: Invalid content key for module: '{content_key}'")
        return
    modules = COURSE_DATA.get("modules", [])
    if module_num < 1 or module_num > len(modules):
        print(f"ERROR: Module {module_num} not found. Available: 1-{len(modules)}")
        return
    module = modules[module_num - 1]
    items = module.get(content_key, [])
    mod_path = module.get("path", "N/A")
    print(f"\nMODULE {module_num} — {content_key.upper()}")
    print("—" * 50)

    if video_final is not None:
        process_module_video_final(module, items, item_index, video_final)
        return
        
    if content_key == "videos":
        if 'Image' in content_type: 
            process_module_images(module, items, item_index, slides_list)
        if 'Audio' in content_type: 
            process_module_audio(module, items, item_index, slides_list)
        if 'pptx' in content_type: 
            process_module_pptx(module, items, item_index)
        if 'Video' in content_type: 
            process_module_video(module, items, item_index, slides_list)

            
        


# ==============================================================
# 6. ГОЛОВНА ФУНКЦІЯ
# ==============================================================
def display_content(
    module_num: Optional[int] = None, #номер модуля
    content_key: Optional[str] = None, #ключ контенту (Video)
    item_index: Optional[int] = None, #порядковий номер контенту. 
    exam: bool = False, #екзамен
    slides_list: Optional[List] = [], # список складів по яким потрібно зробити зміну []-всі слайди
    image_intro: Optional[str] = None, #  Inrtro or Outro створення картинки початковий чи кінцевий
    video_final: Optional[str] = None, #  Inrtro, Outro, Final створення картинки початковий чи кінцевий 
    content_type: Optional[List] = ['Image', 'Audio', 'pptx', 'Video'], #що ми створюємо   
    
) -> None:
    if exam:
        if content_key is not None:
            exam_content_manager(content_key, item_index)
        else:
            print(f"COURSE: {COURSE_DATA.get('course_name')} — FINAL EXAM")
            print("=" * 80)
            for key in EXAM_CONTENT_KEYS:
                pass
                # exam_content_manager(key)
    elif image_intro is not None:
        process_intro_videos(image_intro)
    elif video_final is not None:      
        if module_num is not None and content_key is not None:
            content_manager(module_num, content_key, item_index=item_index, video_final=video_final)
        else:
            modules = COURSE_DATA.get("modules", [])
            print(f"COURSE: {COURSE_DATA.get('course_name')} | {len(modules)} MODULES")
            print("=" * 80)
            for mod_idx in range(1, len(modules) + 1):
                content_manager(mod_idx, "videos", item_index=item_index, video_final=video_final)
                print()
    
    else:
        if module_num is not None and content_key is not None:
            content_manager(module_num, content_key, item_index=item_index, slides_list=slides_list, content_type=content_type)
        elif module_num is None and content_key is None:
            modules = COURSE_DATA.get("modules", [])
            print(f"COURSE: {COURSE_DATA.get('course_name')} | {len(modules)} MODULES")
            print("=" * 80)
            for mod_idx in range(1, len(modules) + 1):
                for key in MODULE_CONTENT_KEYS:
                    content_manager(mod_idx, key, item_index=item_index, content_type=content_type)
                print()
        else:
            print("ERROR: Invalid arguments.")
            print("   display_content(module_num, content_key, item_index) → one item")
            print("   display_content(module_num, content_key)             → all in key")
            print("   display_content()                                   → all modules")
            print("   display_content(exam=True, ...)                     → exam only")


# ==============================================================
# ДЕМОНСТРАЦІЯ
# ==============================================================
if __name__ == "__main__":
    import time

    start = time.perf_counter()
    
    # print("DEMO 1: Усі модулі")
    # print("=" * 80)
    # display_content()

    # print("\n" + "=" * 80)
    # print("DEMO 1: створення images для всього курсу")
    # print("=" * 80)
    # display_content(content_type = ['Image'] )

    # print("\n" + "=" * 80)
    # print("DEMO 2: Модуль 1 — Video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, slides_list = [3,], content_type = ['Image'] )

    # print("\n" + "=" * 80)
    # print("DEMO 3: Модуль 1 — Video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1)

    # print("\n" + "=" * 80)
    # print("DEMO 4: Intro & Outro")
    # print("=" * 80)
    # display_content(image_intro = "Intro")
    # display_content(image_intro = "Outro")
    
    # print("\n" + "=" * 80)
    # print("DEMO 5: Модуль 1 — Audio 1 ")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, slides_list = [], content_type = ['Audio'] )

    # print("\n" + "=" * 80)
    # print("DEMO 6: Модуль 1 — pptx 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, content_type = ['pptx'] )
    
    # print("\n" + "=" * 80)
    # print("DEMO 8: Модуль 1 - аудіо для всього модуля")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", content_type = ['Audio'] )

    # print("\n" + "=" * 80)
    # print("DEMO 7: створення аудіо для всього курсу")
    # print("=" * 80)
    # display_content(content_type = ['Audio'] )

    
    # print("\n" + "=" * 80)
    # print("DEMO 7: pptx всього курсу")
    # print("=" * 80)
    # display_content(content_type = ['pptx'] )

    # print("\n" + "=" * 80)
    # print("DEMO 10: Модуль 1 — video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, content_type = ['Video'], slides_list = [3] )

    # print("\n" + "=" * 80)
    # print("DEMO 11: Модуль 1 — video для всього модуля")
    # print("=" * 80)
    # display_content(module_num = 3, content_key = "videos", item_index = 3, content_type = ['Video'], )

    # print("\n" + "=" * 80)
    # print("DEMO 12: video cлайдів для всього курса")
    # print("=" * 80)
    # display_content(content_type = ['Video'],)

    # print("\n" + "=" * 80)
    # print("DEMO 14: Модуль 1 — video 1")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", item_index = 1, video_final = "Final") # Intro, Outro, Final

    # print("\n" + "=" * 80)
    # print("DEMO 15: Модуль 1 — video для всього модуля")
    # print("=" * 80)
    # display_content(module_num = 1, content_key = "videos", video_final = "Final")

    print("\n" + "=" * 80)
    print("DEMO 16: video для всього курса")
    print("=" * 80)
    display_content(video_final = "Final") # Intro, Outro, Final
    

    elapsed = time.perf_counter() - start
    minutes = int(elapsed // 60)
    seconds = elapsed % 60
    
    print(f"Elapsed time: {minutes} min {seconds:.2f} sec")


DEMO 16: video для всього курса
COURSE: DataScience | 3 MODULES

MODULE 1 — VIDEOS
——————————————————————————————————————————————————
Creating video content: Video 1: What is Data Science?
Creating: Final
Rendering final video → output_DataScience/DataScience/module1/video1/video.mp4
Moviepy - Building video output_DataScience/DataScience/module1/video1/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_DataScience/DataScience/module1/video1/video.mp4



Moviepy - Done !
Moviepy - video ready output_DataScience/DataScience/module1/video1/video.mp4
Subtitles saved (UTF-8 with BOM) → output_DataScience/DataScience/module1/video1/video.srt
Subtitles saved → output_DataScience/DataScience/module1/video1/video.srt
Creating video content: Video 2: The Data Science Workflow
Creating: Final
Rendering final video → output_DataScience/DataScience/module1/video2/video.mp4
Moviepy - Building video output_DataScience/DataScience/module1/video2/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_DataScience/DataScience/module1/video2/video.mp4



Moviepy - Done !
Moviepy - video ready output_DataScience/DataScience/module1/video2/video.mp4
Subtitles saved (UTF-8 with BOM) → output_DataScience/DataScience/module1/video2/video.srt
Subtitles saved → output_DataScience/DataScience/module1/video2/video.srt
Creating video content: Video 3: Python Essentials for Data Science
Creating: Final
Rendering final video → output_DataScience/DataScience/module1/video3/video.mp4
Moviepy - Building video output_DataScience/DataScience/module1/video3/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready output_DataScience/DataScience/module3/video1/video.mp4
Subtitles saved (UTF-8 with BOM) → output_DataScience/DataScience/module3/video1/video.srt
Subtitles saved → output_DataScience/DataScience/module3/video1/video.srt
Creating video content: Video 2: Regression & Classification
Creating: Final
Rendering final video → output_DataScience/DataScience/module3/video2/video.mp4
Moviepy - Building video output_DataScience/DataScience/module3/video2/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_DataScience/DataScience/module3/video2/video.mp4



Moviepy - Done !
Moviepy - video ready output_DataScience/DataScience/module3/video2/video.mp4
Subtitles saved (UTF-8 with BOM) → output_DataScience/DataScience/module3/video2/video.srt
Subtitles saved → output_DataScience/DataScience/module3/video2/video.srt
Creating video content: Video 3: Advanced ML Overview
Creating: Final
Rendering final video → output_DataScience/DataScience/module3/video3/video.mp4
Moviepy - Building video output_DataScience/DataScience/module3/video3/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_DataScience/DataScience/module3/video3/video.mp4



t:  29%|██▉       | 3132/10626 [00:19<00:47, 157.64it/s, now=None]